### **奖励设计**


核心文件位于`feature/reward_process.py`中，需要结合特征处理进行设计，否则出现agent无法感知具体奖励的情况。为了更加清晰框架的奖励设计以及其的工作流程，下面首先分析原本框架的具体流程以及其中关键的数据传输。

#### **整体结构**

训练主循环 -> Agent -> GameRewardManager

1. 训练循环拿到环境给的`obervation[i]`，其中包含：
    - `frame_state`：本帧的完整状态
2. Agent调用：
```python
reward = agent.reward_manager.result(observation[i]["frame_state"])
```

3. `result()`返回本步奖励字典(每个子项 + 合计)，供PPO算法记于学习

#### **数据流**

1. **输入数据结构**：输入环境的`frame_states`

2. **输出数据结构**：`result()`的返回值
```python
{
  "forward": float,
  "tower_hp_point": float,
  "hero_hp_point": float,
  "gold_point": float,
  "minion_push_depth": float,
  "kill_event": float,
  "death_event": float,
  "tower_danger": float,
  "dive_no_minion": float,
  "grass_engage": float,
   # 汇总分：按 GameConfig.REWARD_WEIGHT_DICT 做线性加权求和
  "reward_sum": float
}
```


#### **关键类**

`GameRewardManager` \
该类把每一帧的`frame_state`转成本步可以学习奖励字典

**关键成员**

- `m_main_calc_frame_map: dict[str, RewardStruct]` \
当前“己方阵营”视角的各奖励条目“帧值缓存”（含上一帧值与当前帧值）。
- `m_enemy_calc_frame_map: dict[str, RewardStruct]` \
敌方阵营视角的同构缓存
- `ABS_NAMES: set[str]` \
指明哪些条目在`get_reward`时按非零和/非差分使用
- `EVENT_NAMES: set[str]` \
指明哪些条目按当帧敌我差使用
- `TIME_SCALE_ARG: int`
时间衰减超参

`RewardStruct` \
保存每个条目在“本阵营视角”的上一帧值与当前帧值，方便在后序的奖励计算中进行帧间差分

**关键字段**

```python
class RewardStruct:
    last_frame_value: float  # t-1 帧的度量
    cur_frame_value:  float  # t 帧的度量
```

#### **关键函数**

`result(frame_state) -> dict`

入口调用函数，整合调用顺序：
1. `frame_data_process(frame_state)`
- 用于产出/更新`m_main_calc_frame_map`、`m_enemy_calc_frame_map`中每个条目的`last_frame_value/cur_frame_value`

2. `get_reward(m_main_calc_frame_map, m_enemy_calc_frame_map, frame_state)`
- 把帧信息变成每一步奖励：
  - 对`ABS_NAMES`：直接用我方`cur_frame_value`
  - 对`EVENT_NAMES`：用（我方当帧 − 敌方当帧）
  - 对其他条目：用双方差的帧间差分：
  `[(我方_cur - 敌方_cur) - (我方_last - 敌方_last)]`
  - 若启用时间衰减：乘`0.6 ** (frameNo / TIME_SCALE_ARG)`
  - 按权重表`GameConfig.REWARD_WEIGHT_DICT`做线性加权，得到`reward_sum`

3. 返回奖励字典

---

`frame_data_process(frame_state)`

把一帧输入拆成己方视角/敌方视角的帧度量缓存：
1. 识别阵营
  - 从`frame_state.hero_states`找出敌方/己方英雄
2. 刷新两套缓存
  - 调用`set_cur_calc_frame_vec(self.m_main_calc_frame_map, frame_state, main_camp)`
  使得每个条目的`cur_frame_value`和`last_frame_value`计算
  - 同样针对敌方的情况进行一致的流程
3. 完成后两套Map已经更新完成，送入`get_reward`

核心数据：`self.m_main_calc_frame_map`和`self.m_enemy_calc_frame_map`

---

`set_cur_calc_frame_vec(cul_calc_frame_map, frame_data, camp)`

一个阵营的帧值填充器
- 输入：
  - `cul_calc_frame_map: dict[str, RewardStruct]`将会刷新的一套条目表
  - `frame_data: dict`重要的`frame_state`
  - `camp: str`该视角的阵营，注意阵营形式为`PLAYERCAMP_2/PLAYERCAMP_1`

- 流程：
  - 针对每个条目，将旧的`cur_frame_value`转移到`last_frame_value`
  - 计算并写入`cur_frame_value`

- 输出：
  - 最终完成对`self.m_main_calc_frame_map`和`self.m_enemy_calc_frame_map`的更新

---

`get_reward(main_map, enemy_map, frame_state) -> dict`

策略整合器

- 按条目组(`ABS_NAMES、EVENT_NAMES`)选择不同的规则，将帧度量组合成本步最终奖励
  - ABS：直接用`main_map[name].cur_frame_value`
  - 事件：`main_cur - enmey_cur`
  - 普通密集量(零和 + 差分)：`(main_cur - enemy_cur) - (main_last - enemy_last)`
- 可选时间衰减
- 加权汇总
- 返回每个条目本步奖励 + reward_sum字典


In [1]:
# 下面是整体流程的伪代码
class GameRewardManager:
    def __init__(self, config):
        self.m_main_calc_frame_map = {name: RewardStruct() for name in ALL_NAMES}
        self.m_enemy_calc_frame_map = {name: RewardStruct() for name in ALL_NAMES}
        self.ABS_NAMES = {...}
        self.EVENT_NAMES = {...}
        self.TIME_SCALE_ARG = config.TIME_SCALE_ARG
        self.weights = GameConfig.REWARD_WEIGHT_DICT

    def result(self, frame_state) -> dict:
        # 1) 用两套视角把“本帧度量”准备好
        self.frame_data_process(frame_state)

        # 2) 把“帧度量”转换为“本步奖励”，并做加权
        reward_dict = self.get_reward(self.m_main_calc_frame_map,
                                      self.m_enemy_calc_frame_map,
                                      frame_state)
        return reward_dict

    def frame_data_process(self, frame_state):
        main_camp, enemy_camp = identify_camps(frame_state)
        self.set_cur_calc_frame_vec(self.m_main_calc_frame_map, frame_state, main_camp)
        self.set_cur_calc_frame_vec(self.m_enemy_calc_frame_map, frame_state, enemy_camp)

    def set_cur_calc_frame_vec(self, cul_calc_frame_map, frame_state, camp):
        for name, st in cul_calc_frame_map.items():
            st.last_frame_value = st.cur_frame_value
            st.cur_frame_value  = compute_frame_metric_for(name, frame_state, camp)  # 只填“帧值”

    def get_reward(self, main_map, enemy_map, frame_state) -> dict:
        per_name = {}
        for name in ALL_NAMES:
            if name in self.ABS_NAMES:
                value = main_map[name].cur_frame_value
            elif name in self.EVENT_NAMES:
                value = main_map[name].cur_frame_value - enemy_map[name].cur_frame_value
            else:
                value = ((main_map[name].cur_frame_value - enemy_map[name].cur_frame_value)
                        -(main_map[name].last_frame_value - enemy_map[name].last_frame_value))

            if self.TIME_SCALE_ARG > 0:
                t = frame_state.get("frameNo", 0)
                value *= 0.6 ** (t / self.TIME_SCALE_ARG)

            per_name[name] = value

        reward_sum = sum(self.weights.get(n, 0.0) * v for n, v in per_name.items())
        per_name["reward_sum"] = reward_sum
        return per_name


#### **关于阵营信息**

代码中存在部分阵营信息提取不太理想/相对繁琐的问题，下面有一个比较好的阵营信息的提取方式，具体如下：
```python
for hero in frame_data["hero_states"]:
    if hero["player_id"] == self.main_hero_player_id:
        main_camp = hero["actor_state"]["camp"]
        self.main_hero_camp = main_camp
    else:
        enemy_camp = hero["actor_state"]["camp"]
```

注意提取得到的`main_camp`和`enemy_camp`均为`PLAYERCAMP_1/PLAYERCAMP_2`

### **当前设计设计-版本1**


注意这里`1 step = 6 frame`，按照逐帧取值，算法侧每步(6帧)再聚合


#### **`forward`前进奖励**

`forward`前进奖励为非零和的连续值，只针对当前帧的情况计算即可

1. 取出三点坐标：`main_tower_pos`，`enemy_tower_pos`，`hero_pos`

2. 计算几个距离`dist_hero2emy = dist(hero, enemy_tower)`，`dist_main2emy = dist(main_tower, enemy_tower)`

3. 奖励为比值`base`，并且远离时不给奖励(同时这里约束了一个下届$base \ge 0$)

4. 最后将`baes`乘上归一化的血量得到最终的前进奖励

具体计算函数：
```python
def calculate_forward(self, main_hero, main_tower, enemy_tower):
    main_tower_pos = (main_tower["location"]["x"], main_tower["location"]["z"])
    enemy_tower_pos = (enemy_tower["location"]["x"], enemy_tower["location"]["z"])
    hero_pos = (
        main_hero["actor_state"]["location"]["x"],
        main_hero["actor_state"]["location"]["z"],
    )
    forward_value = 0
    dist_hero2emy = math.dist(hero_pos, enemy_tower_pos)
    dist_main2emy = max(math.dist(main_tower_pos, enemy_tower_pos), 1e-6)
    base = (dist_main2emy - dist_hero2emy) / dist_main2emy
    base = max(0.0, base)  # 远离敌塔不奖励
    hp = float(main_hero["actor_state"]["hp"])
    mx = max(float(main_hero["actor_state"]["max_hp"]), 1.0)
    hp_scale = hp / mx
    return base * hp_scale  # 或者直接 return base
```

#### **`tower_hp_point`塔血比例**

塔血比例需要同时计算己方和敌方的血量，所以为零和项


使用辅助函数得到塔血量比例，
```python
def _hp_ratio(u):
    if not isinstance(u, dict):
        return 0.0
    hp = float(u.get("hp", 0.0))
    mx = float(u.get("max_hp", 0.0))
    return hp / mx if mx > 0 else 0.0

hero_hp_ratio = _hp_ratio((main_hero or {}).get("actor_state") or {})
my_tower_hp_ratio = _hp_ratio(main_tower)
```
